In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
import pandas as pd
df = pd.read_csv("spam.csv")
df.head()

In [ ]:
df["Category"].value_counts()

### It is imbalance dataset so use down sampleing - pick random 747 email from ham catergory

In [ ]:
747/4825

* Making seprate dataframe of spam and ham

In [ ]:
df_spam = df[df["Category"]=="spam"]
df_spam.shape

In [ ]:
df_ham = df[df["Category"]=="ham"]
df_ham.shape

In [ ]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

In [ ]:
df_balanced = pd.concat([df_spam,df_ham_downsampled])
df_balanced.shape

In [ ]:
df_balanced['Category'].value_counts()

In [ ]:
df_balanced.head()

In [ ]:
df_balanced['spam'] = df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df_balanced["Message"],df_balanced["spam"],stratify=df_balanced["spam"])

In [ ]:
len(x_train),len(x_test)

In [ ]:
bert_processor = hub.KerasLayer(hub.load("preprocessor"))
bert_encoder = hub.KerasLayer(hub.load("archive"))

In [ ]:
def get_embedding(sentences):
    preprocessed_sentences = bert_processor(sentences)
    result = bert_encoder(preprocessed_sentences)
    return result["pooled_output"]
get_embedding(["nice movie indeed","hello how are you"])

In [ ]:
e = get_embedding(["grapes","mango","banana","elon musk","bill gates"])
e

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity([e[0]],[e[1]]))
print(cosine_similarity([e[2]],[e[1]]))

### Creating a  functional  model 

In [ ]:
# Bert Layers
text_input = tf.keras.layers.Input(shape=(),dtype=tf.string,name='text')
preprocessed_input = bert_processor(text_input)
output = bert_encoder(preprocessed_input)

# Neural Network layer
l = tf.keras.layers.Dropout(0.1,name='dropout')(output['pooled_output'])
l = tf.keras.layers.Dense(1,activation='sigmoid',name='output')(l)

#construct a final model
model = tf.keras.Model(inputs=[text_input],outputs=[l])

In [ ]:
model.summary()

In [ ]:
METRICS=[
    tf.keras.metrics.BinaryAccuracy(name="accuracy"),
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name="recall")
]

model.compile(optimizer='adam',loss ='binary_crossentropy',metrics=METRICS)

In [ ]:
model.fit(x_train,y_train,epochs=10)

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
y_predicted = model.predict(x_test)

In [ ]:
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np
                       #condition , if true then , else 
y_predicted = np.where(y_predicted>0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
cm = confusion_matrix(y_test,y_predicted)
cm

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
sn.heatmap(cm,annot=True,fmt='d')
plt.xlabel("predicted")
plt.ylabel("Truth")

In [ ]:
print(classification_report(y_test,y_predicted))